# Traffic Detection using Image Classification and Object Detection
 Two different model architectures would be used to train the custom model. For image classification and object detection, respectively, 
 the **EfficientNet** and **YOLO** models were chosen for their reliability and fast inference times, effective for real-time processing.

### Dataset
- COCO 2017 https://cocodataset.org/ 
- Traffic-Netv1 https://github.com/OlafenwaMoses/Traffic-Net/releases

Importing COCO 

In [3]:
import fiftyone as fo
import fiftyone.zoo as foz

In [40]:
# Define the classes and maximum samples per class
classes = ["person", "bicycle", "motorcycle", "bus", "truck", "car"]

# Load COCO dataset with annotations filtered for classes
detection_dataset = foz.load_zoo_dataset(
    "coco-2017",
    splits=["train","validation"],
    label_types=["detections"],
    classes=classes,
    include_id=False,
    include_license=False,
    only_matching=True,
    seed=12,
    max_samples=5000, 
    progress=True
)


 100% |██████|    1.9Gb/1.9Gb [1.6m elapsed, 0s remaining, 26.3Mb/s]        
Extracting annotations to '/Users/bualoydgreat/fiftyone/coco-2017/raw/instances_train2017.json'
 100% |████████████████| 5000/5000 [18.0m elapsed, 0s remaining, 2.9 images/s]      
Writing annotations for 5000 downloaded samples to '/Users/bualoydgreat/fiftyone/coco-2017/train/labels.json'
Found annotations at '/Users/bualoydgreat/fiftyone/coco-2017/raw/instances_val2017.json'
Only found 2968 (<5000) samples matching your requirements
 100% |████████████████| 2968/2968 [11.0m elapsed, 0s remaining, 3.0 images/s]      
Writing annotations for 2968 downloaded samples to '/Users/bualoydgreat/fiftyone/coco-2017/validation/labels.json'
Dataset info written to '/Users/bualoydgreat/fiftyone/coco-2017/info.json'
Loading 'coco-2017' split 'train'
 100% |███████████████| 5000/5000 [9.2s elapsed, 0s remaining, 546.0 samples/s]       
Loading 'coco-2017' split 'validation'
 100% |███████████████| 2968/2968 [5.6s elapsed, 

In [ ]:
session = fo.launch_app(detection_dataset)

In [43]:
# Export the filtered dataset to COCO format
export_dir = "/Users/bualoydgreat/PERSONAL-TEMPS/optimized-traffic-light-computer-vision/dataset/yolo"

# The splits to export
splits = ["train", "validation"]

# Export the splits
for split in splits:
    split_view = detection_dataset.match_tags(split)
    split_view.export(
        export_dir=export_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        label_field="ground_truth",
        split=split,
        classes=classes,
    )

 100% |███████████████| 5000/5000 [8.0s elapsed, 0s remaining, 687.3 samples/s]       
Directory '/Users/bualoydgreat/PERSONAL-TEMPS/optimized-traffic-light-computer-vision/dataset/yolo' already exists; export will be merged with existing files
 100% |███████████████| 2968/2968 [4.4s elapsed, 0s remaining, 716.8 samples/s]      


Importing Traffic-Net v1

In [47]:
!curl -L https://github.com/OlafenwaMoses/Traffic-Net/releases/download/1.0/trafficnet_dataset_v1.zip > trafficnet.zip
!unzip trafficnet.zip
#!sudo rm trafficnet.zip
#!sudo mv trafficnet_dataset_v1 /dataset/trafficnet

## Object Detection
Using Ultralytics YOLOv8 to train the model.

### Training

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

# Train the model using M1/M2
results = model.train(data='dataset.yaml', epochs=100, imgsz=640, device='mps')